In [1]:
from rouge import Rouge
import glob
import os
import re
import pandas as pd

#goldSummaries = glob.glob(r'..data/raw/OpinosisDataset1.0_0/summaries-gold/*/')
gold_summaries = os.walk(r'../data/raw/OpinosisDataset1.0_0/summaries-gold')
gold_list = next(gold_summaries)[1]

In [9]:
def rouge_evaluation(summaryfiles_dir, summaryDir):
    summaryFiles = glob.glob(summaryfiles_dir)
    rouge_1_scores_list = []
    rouge_2_scores_list = []
    rouge_l_scores_list = []
    for h in summaryFiles:
        foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
        foldername = (foldername_search.group()).split('.')[0]
        with open(h, 'r') as f:
            hypothesis = f.read()
            if foldername in gold_list:
                files = glob.glob(summaryDir+foldername+'/*')
                foldername = ' '.join(foldername.split('_'))
                for r in files:
                    with open(r, 'r') as f:
                        reference = f.read()
                        rouge = Rouge()
                        scores = rouge.get_scores(hypothesis, reference)[0]
                        rouge_1_scores_list.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
                        rouge_2_scores_list.append(pd.DataFrame(scores['rouge-2'], index=[foldername]))
                        rouge_l_scores_list.append(pd.DataFrame(scores['rouge-l'], index=[foldername]))
                        rouge_1_df = pd.concat(rouge_1_scores_list)
                        rouge_2_df = pd.concat(rouge_2_scores_list)
                        rouge_l_df = pd.concat(rouge_l_scores_list)
    return rouge_1_df, rouge_2_df, rouge_l_df

In [10]:
textrank_rouge_1, textrank_rouge_2, textrank_rouge_l = rouge_evaluation(r'../data/processed/textrank/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [11]:
lexrank_rouge_1, lexrank_rouge_2, lexrank_rouge_l = rouge_evaluation(r'../data/processed/lexrank/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [12]:
opinosis_rouge_1, opinosis_rouge_2, opinosis_rouge_l = rouge_evaluation(r'../data/processed/opinosis/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [14]:
textrank_rouge_1.describe().round(3).to_csv('textrank_rouge_1_stats.csv')
textrank_rouge_2.describe().round(3).to_csv('textrank_rouge_2_stats.csv')
textrank_rouge_l.describe().round(3).to_csv('textrank_rouge_l_stats.csv')

In [15]:
lexrank_rouge_1.describe().round(3).to_csv('lexrank_rouge_1_stats.csv')
lexrank_rouge_2.describe().round(3).to_csv('lexrank_rouge_2_stats.csv')
lexrank_rouge_l.describe().round(3).to_csv('lexrank_rouge_l_stats.csv')

In [16]:
opinosis_rouge_1.describe().round(3).to_csv('opinosis_rouge_1_stats.csv')
opinosis_rouge_2.describe().round(3).to_csv('opinosis_rouge_2_stats.csv')
opinosis_rouge_l.describe().round(3).to_csv('opinosis_rouge_l_stats.csv')

In [17]:
textrank_rouge_1.shape

(238, 3)

In [18]:
lexrank_rouge_1.shape

(238, 3)

In [19]:
opinosis_rouge_1.shape

(238, 3)

In [20]:
from scipy.stats import f_oneway